In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
from __future__ import division
import gdal
import numpy as np
import matplotlib.pyplot as plt
import json
import pyproj as pj

In [3]:
dem_dataset = gdal.Open('../data/dem/s_tibet_utm45n_500m_clip.tif')

In [4]:
dem_transform = {'upper_left_x': dem_dataset.GetGeoTransform()[0],
                 'x_res_m' : dem_dataset.GetGeoTransform()[1],
                 'x_rotation': dem_dataset.GetGeoTransform()[2],
                 'upper_left_y': dem_dataset.GetGeoTransform()[3],
                 'y_rotation': dem_dataset.GetGeoTransform()[4],
                 'y_res_m': dem_dataset.GetGeoTransform()[5],
                 'n_cols': dem_dataset.RasterXSize,
                 'n_rows': dem_dataset.RasterYSize}

dem_transform['y_res_m'] *= -1 # correct for the upper-left origin thing

print(dem_transform)

{'n_rows': 2548, 'x_res_m': 423.895245806536, 'upper_left_x': -345300.6406560004, 'x_rotation': 0.0, 'y_rotation': 0.0, 'y_res_m': 423.895245806536, 'n_cols': 3763, 'upper_left_y': 3650189.109228576}


In [5]:
dem_transform['east_min'] = dem_transform['upper_left_x']

dem_transform['east_max'] = (dem_transform['x_res_m'] * dem_transform['n_cols']
                             + dem_transform['east_min'])

dem_transform['north_max'] = dem_transform['upper_left_y']

dem_transform['north_min'] = (-1 * dem_transform['y_res_m'] * dem_transform['n_rows']
                              + dem_transform['north_max'])

In [6]:
wgs84 = pj.Proj(init='epsg:4326')
utm41n = pj.Proj(init='epsg:32641')

dem_transform['lon_min'], dem_transform['lat_min'] = pj.transform(utm41n, wgs84,
                                                            dem_transform['east_min'], 
                                                            dem_transform['north_min'])

dem_transform['lon_max'], dem_transform['lat_max'] = pj.transform(utm41n, wgs84,
                                                            dem_transform['east_max'], 
                                                            dem_transform['north_max'])